<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-Control-Sample" data-toc-modified-id="Load-Control-Sample-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load Control Sample</a></span></li><li><span><a href="#Run-ESASky" data-toc-modified-id="Run-ESASky-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Run ESASky</a></span></li><li><span><a href="#Construct-Data-Sample" data-toc-modified-id="Construct-Data-Sample-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Construct Data Sample</a></span></li><li><span><a href="#Visual-Inspection" data-toc-modified-id="Visual-Inspection-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Visual Inspection</a></span></li><li><span><a href="#Clustering-Analysis" data-toc-modified-id="Clustering-Analysis-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Clustering Analysis</a></span></li><li><span><a href="#Compare-to-Control" data-toc-modified-id="Compare-to-Control-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Compare to Control</a></span></li><li><span><a href="#Save-HDBSCAN-slected-Output" data-toc-modified-id="Save-HDBSCAN-slected-Output-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Save HDBSCAN slected Output</a></span></li></ul></div>

In [1]:
# Python Basics ========================
import warnings, itertools, sys, os
warnings.filterwarnings('ignore', category=UserWarning)
sys.path.append('../library_pangaia_2019_11_10/')

# Custom ===============================
from lib_utils       import LibUtils       as Utils
from lib_gaia_query  import LibGaiaQuery   as GaiaQuery
from lib_interactive import LibInteractive as Interactive
from lib_cluster     import LibCluster     as Cluster
from lib_plotters    import LibPlotters    as Plotters
from lib_compare     import LibCompare     as Compare

# PyEsasKy =============================
from pyesasky.pyesasky import ESASkyWidget

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


In [2]:
# SetUp Analysis Steps =================================
step_1 = True   # Load Control Sample
step_2 = True   # ESASky Plot
step_3 = True   # ADQL to Gaia DR12
step_4 = True   # Interactive / Dynamical Gaia Data exploration
step_5 = True   # Clustering

# Load Control Sample

In [ ]:
# Load Control Sample ==================================
if step_1:
    control = Utils(color = 'magenta', label = 'Sample Control')
    control.read_catalogue()
    print()
    print(f'Showing {control.label} first 5 rows:')
    display(control.cat[0:5])

    # Examine Velocity & Histograms ====================
    figures  = Plotters()
    figures.load_gaia_cat(control.cat)
    figures.plot_2d_and_hist(col_hist = 'distance', label = control.label,
                             color_2d = control.color, color_hist = control.color )    

Sample Catalogues in current directory: ==============================================

* test_r1.5_2.4_4.1_scl_standard.vot
* test_r0.75_2.4_4.1_scl_standard.vot
* test_r1.25_2.4_4.1_minsamp_1_mCls_19_cl_0.vot
* test_r1.25_2.5_4.0.vot
* test_r1.0_2.5_4.0_minsamp_1_mCls_19_cl_0.vot
* r1.25_2.3_4.2_scl_standard.vot
* test_IAA.vot
* test_r1.0_2.4_4.1_minsamp_1_mCls_19_cl_0.vot
* test_r1.25_2.4_4.1_scl_standard.vot
* simbad_new.vot
* test_r1.0_2.5_4.1_scl_standard.vot
* simbad_new_test.vot
* test_dani_scl_standard.vot
* sample_control_IC_348.vot
* test_r1.25_2.4_4.1.vot
* test_r1.25_2.5_4.0_scl_standard.vot
* 10_make_photometry_tb_1.vot
* test_r1.0_2.4_4.1.vot
* test_r1.25_2.5_4.0_minsamp_1_mCls_20_cl_0.vot
* r1.25_2.3_4.2_minsamp_1_mCls_20_cl_0.vot
* test_r1.75_2.4_4.1_scl_standard.vot
* test_r1.0_2.3_4.3_scl_standard.vot
* test_r1.25_2.4_4.1_minsamp_1_mCls_20_cl_0.vot
* test_r0.5_2.4_4.1_scl_standard.vot
* test_r1.0_2.4_4.1_scl_standard.vot
* r1.25_2.3_4.2.vot
* test_r0.75_2.4_4.1_minsa

# Run ESASky

In [ ]:
# Run ESASKY ===========================================
if step_2:
    esasky = ESASkyWidget()
    display(esasky)

In [ ]:
# Show Control Sample in ESASky ========================
if step_1 and step_2:
    figures_0  = Plotters()
    figures_0.load_gaia_cat(control.cat)
    figures_0.send_to_ESASky(esasky, background='WISE', color=control.color, catalogueName = control.label)

# Construct Data Sample

In [ ]:
# Create Gaia Sample ===================================
if step_3:
    sample_queried = GaiaQuery()
    if step_1:
        control.print_cone_properties()
        ra  = control.ADQL_pars['ra'].value
        dec = control.ADQL_pars['dec'].value
        sample_queried.set_cone_search_pars(ra = ra, dec = dec)
    else:
        sample_queried.set_cone_search_pars()
    sample_queried.run_cone_search(quality_par_SN  = '10', quality_par_vis = '7',quality_par_ruwe = '1.40')

    print()
    sample_gaia = Utils(color = 'grey', label = sample_queried.sample_name)
    sample_gaia.read_catalogue(sample_queried.cat, save_sample = True, print_vrad = True)

    print('Showing first 5 rows of Analysis Sample')
    display(sample_gaia.cat[0:5])

In [ ]:
# Show output in ESASky ================================
if step_2:
    figures  = Plotters()
    figures.load_gaia_cat(sample_gaia.cat)
    figures.send_to_ESASky(esasky, background='WISE', color='white', radius = sample_gaia.ADQL_pars['radius'].value, catalogueName = 'Analysis Sample')

# Visual Inspection

In [ ]:
# Examine astrometry ===================================
if step_4:
    i_figures = Interactive()
    i_figures.load_gaia_cat(sample_gaia.cat)
    if step_1:
            i_figures.load_control_obj(control)
    i_figures.explore_and_select(opacity = 0.5, border_width = 1, color_def = 'black', color_high = 'cyan')
    display(i_figures.dashboard)

In [ ]:
# Show 3D distribution =================================
if step_2 and step_4:
    if hasattr(i_figures, 'cat_subsamp'):
        print('Plotting Subsample in ESASky')
        figures.add_catalogue_to_ESASky(esasky, i_figures.cat_subsamp, color='cyan', catalogueName='SubSamp', radius = sample_gaia.ADQL_pars['radius'].value)

if step_4:
    i_figures.show_3D_space(width = 1200, height=700, size = 5, color_def = 'black')
    display(i_figures.fig_3D)

# Clustering Analysis

In [ ]:
# Scale before clustering ==============================
if step_5:
    data_clustering = Cluster()
    data_clustering.read_data_obj(sample_gaia, scl_features = ['X_gal', 'Y_gal', 'Z_gal', 'pmdec', 'pmra'])

In [ ]:
# Run HDBSCAN through a range of min samples ===========
if step_5:
#     data_clustering.run_multi_hdbscan(verbose = True, show_plot = False, probability = 80, min_samples = 1)
    data_clustering.run_multi_hdbscan(verbose = True, show_plot = False)
    data_clustering.plot_multi_hdbscan_stats(fig_nm = 'default')

In [ ]:
# Show selected HDBSCAN results ========================
if step_5:
    data_clustering.run_hdbscan(probability = data_clustering.probability, min_samples=data_clustering.min_samples, verbose = True)
    data_clustering.plot_hdbscan_clusters(xlim_1=[-5,15], ylim_1 = [-17.5,2.5])
    if step_2:
        data_clustering.esasky_hdbscan_clusters(esasky, radius = sample_gaia.ADQL_pars['radius'].value)

# Compare to Control

In [ ]:
# Cluster Comparison ===================================
if step_1:
    comp = Compare()
    comp.read_control(control, mew = 2)
    comp.read_clusters(data_clustering, verbose = False)
    comp.compare_to_control(verbose = True)
    print()
    
    # Plot Comparison ======================================
    # Define plot XY ranges for panels 1 & 2
    xlim_1 = [sample_gaia.cat['ra'].min(),  sample_gaia.cat['ra'].max()]
    ylim_1 = [sample_gaia.cat['dec'].min(), sample_gaia.cat['dec'].max()]
    xlim_2 = [0.0, 10.0]
    ylim_2 = [-12.5, -2.5]

    comp.plot_clusters_comp(markersize = 16, xlim_1 = xlim_1, ylim_1 = ylim_1, xlim_2 = xlim_2, ylim_2 = ylim_2, fig_nm = 'default')

# Save HDBSCAN slected Output

In [ ]:
if step_5:
    data_clustering.save_cluster()